# Movies Recommender System

이 커널은 학습을 위해 다음 커널을 필사/번역한 내용입니다. [Code](https://www.kaggle.com/rounakbanik/movie-recommender-systems)

이번 notebook에서 몇 개의 추천 시스템 알고리즘(content based, popularity based,
 그리고 collaborative filtering)을 시행해보고, 우리의 최종 추천 시스템 도출을 위한 이 모델들의 ensemble을 개발해보겠다. 

우리는 2개의 MovieLens datasets을 가지고 있다.
* The Full Dataset : 270,000 사용자가 45,000 영화에 매긴 26,000,000 평점과 750,000개의 tag application들이 있다. 1100개의 tag에 대해 1200만 개의 관련성 점수가 있는 tag genome 데이터를 포함한다.
* The Small Dataset : 700명의 유저가 9000개의 영화를 100,000개의 평점과 1300개의 tag application한 것으로 구성했다. 

Small Dataset을 이용하여 간단한 Recommender system을 개발하겠다.


In [1]:
#notebook을 실행한 브라우저에서 바로 그림을 볼 수 있게 해줌
%matplotlib inline 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from ast import literal_eval
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet
from surprise import Reader, Dataset, SVD
# evaluate가 cross_valiate로 대체 
from surprise.model_selection import cross_validate


import warnings; warnings.simplefilter('ignore')

## Simple Recommender / 유명한 것들만 추천
Simple Recommender는 모두에게 영화의 인기와 장르에 따른 일반적인 추천을 제공한다.  
이것의 아이디어는 더 유명하고, 더 비평가들의 찬사를 받는 영화가 대중들이 좋아할 확률이 높다는 것에 기반을 두고 있습니다.  
이 모델은 사용자에 따라 개인화된 추천을 제공하지는 않습니다.


이 모델의 구현은 간단합니다.  
우리가 해야할 일은 인기와 평점을 기반으로 정렬하고 상위 영화를 표시하는 것입니다. 추가로 특정 장르의 상위 영화를 얻기 위해 장르 인수를 전달할 수 있다.

Simple Recommender의 흐름
1. 전체 상위
    1. 상위 항목을 위한 가중치 공식을 세움
    2. 그것을 기반으로 상위 항목 골라냄

2. 장르나, 감독에 해당하는 상위
    1. 원하는 항목에 해당하는 정보들만 골라서 다시 작업
        1. 우리가 공모전에서 이 작업을 수행할 것
        
        
* 그렇다면 내가 나중에 해야할 것은?
    * 어떠한 가중치로 추천할 것인가
        * 조회수/공고일자, 상금, 이 전에 대회가 열린적이 있는가
        * 어떤 항목들로 고를 것인가

In [2]:
# 데이터 불러오기
md = pd.read_csv('../input/the-movies-dataset/movies_metadata.csv')
md.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [3]:
# 결측치 확인
md['genres'].isnull().sum()

0

In [4]:
# literal_eval의 기능 설명
type(md['genres'][0]), type(literal_eval(md['genres'][0])), md['genres'][0]

(str,
 list,
 "[{'id': 16, 'name': 'Animation'}, {'id': 35, 'name': 'Comedy'}, {'id': 10751, 'name': 'Family'}]")

https://stackoverflow.com/questions/60042516/how-do-i-figure-out-what-this-code-is-doing

In [5]:
# literal_eval : type 변환 시켜주는 것, string -> dict, eval은 두려움
# isinstance(x, list) : x가 list인지 확인해봅니다.
md['genres'] = md['genres'].fillna('[]').apply(literal_eval).apply(lambda x : [i['name'] for i in x] if isinstance(x, list) else [])

TMDB Ratings를 사용하여 Top Movies Chart를 만들었다.  
IMDB의 가중치 공식을 사용하여 우리의 차트를 구성했다. 수학적으로, 다음과 같이 표현된다:  
Weight Rating(WR) = $(\frac{v}{v+m}\cdot{R})+(\frac{m}{v+m}\cdot{C})$,
* v : 영화의 vote 수
* m : chart에 등록되기 위한 최소한의 vote 수
* R : 영화 평점의 평균
* C : 전체 report에 대한 평균 vote 수

다음 차례는 chart에 등록되기 위한 최소한의 vote 수인 m에 적절한 값을 결정하는 것이다. 우리는 95% 백분위 수를 우리의 컷오프로 사용합니다. 즉, 영화가 차트에 포함되려면 목록에 있는 영화의 95% 이상보다 더 많은 표가 있어야합니다.

전체 Top 250 차트를 작성하고 특성 장르에 대한 차트를 작성하는 기능을 정의합니다.

https://math.stackexchange.com/questions/169032/understanding-the-imdb-weighted-rating-function-for-usage-on-my-own-website

In [6]:
md.shape

(45466, 24)

In [7]:
vote_counts = md[md['vote_count'].notnull()]['vote_count'].astype('int')
vote_averages = md[md['vote_average'].notnull()]['vote_average'].astype('int')
C = vote_averages.mean()
C # 평균적으로 5.2표씩 받음

5.244896612406511

In [8]:
m = vote_counts.quantile(0.95)
m

434.0

In [9]:
# errors='coerce' : 문자열이 속해있어서 오류가 날 경우 강제로 NaT으로 출력
md['year'] = pd.to_datetime(md['release_date'], errors='coerce').apply(lambda x: str(x).split('-')[0] if not(pd.isnull(x)) else np.nan)

In [10]:
qualified = md[(md['vote_count'] >= m) & (md['vote_count'].notnull()) & (md['vote_average'].notnull())][['title', 'year', 'vote_count', 'vote_average', 'popularity', 'genres']]
qualified['vote_count'] = qualified['vote_count'].astype('int')
qualified['vote_average'] = qualified['vote_average'].astype('int')
qualified.shape

(2274, 6)

차트에 선정 되려면 영화는 TMDB에서 최소 434표를 받아야 한다.  
또한 TMDB에 있는 영화의 평균 등급은 5.244점이다.  
2274개의 영화가 차트에 포함될 자격이 있다.

In [11]:
def weighted_rating(x) :
    v = x['vote_count']
    R = x['vote_average']
    return (v/(v+m) * R) + (m/(m+v) * C)

In [12]:
qualified['wr'] = qualified.apply(weighted_rating, axis = 1)

In [13]:
# wr기준으로 상위 250개의 영화만 골라냄
qualified = qualified.sort_values('wr', ascending = False).head(250)

**Top Movies**

In [14]:
qualified.head(15)

,title,year,vote_count,vote_average,popularity,genres,wr
15480,Inception,2010,14075,8,29.108149,"[Action, Thriller, Science Fiction, Mystery, A...",7.917588
12481,The Dark Knight,2008,12269,8,123.167259,"[Drama, Action, Crime, Thriller]",7.905871
22879,Interstellar,2014,11187,8,32.213481,"[Adventure, Drama, Science Fiction]",7.897107
2843,Fight Club,1999,9678,8,63.869599,[Drama],7.881753
4863,The Lord of the Rings: The Fellowship of the Ring,2001,8892,8,32.070725,"[Adventure, Fantasy, Action]",7.871787
292,Pulp Fiction,1994,8670,8,140.950236,"[Thriller, Crime]",7.868660
314,The Shawshank Redemption,1994,8358,8,51.645403,"[Drama, Crime]",7.864000
7000,The Lord of the Rings: The Return of the King,2003,8226,8,29.324358,"[Adventure, Fantasy, Action]",7.861927
351,Forrest Gump,1994,8147,8,48.307194,"[Comedy, Drama, Romance]",7.860656
5814,The Lord of the Rings: The Two Towers,2002,7641,8,29.423537,"[Adventure, Fantasy, Action]",7.851924


우리는 크리스토퍼 놀란의 인셉션, 다크나이트, 인터스텔라 총 3개의 작품이 매우 상위권에 위치함을 알 수 있다. 차트는 TMDB의 유저들이 특별한 장르와 감독에게 강한 편향을 가짐을 나타낸다.  
이제 우리는 특별한 장르를 위한 함수를 구축해볼 것이다. 이것을 위해, 백분위 수 95% 대신 85%로 조건을 완화해보자.

In [15]:
# stack : 들어온 것 부터 쌓음
s = md.apply(lambda x: pd.Series(x['genres']), axis=1).stack().reset_index(level=1, drop=True)
s.name = 'genre'
gen_md = md.drop('genres', axis=1).join(s)

In [16]:
df = gen_md[gen_md['genre'] == 'Comedy']

In [17]:
def build_chart(genre, percentile=0.85):
    df = gen_md[gen_md['genre'] == genre]

    vote_counts = df[df['vote_count'].notnull()]['vote_count'].astype('int')
    vote_averages = df[df['vote_average'].notnull()]['vote_average'].astype('int')
    C = vote_averages.mean()
    m = vote_counts.quantile(percentile)

    qualified = df[(df['vote_count'] >= m) & (df['vote_count'].notnull()
                & (df['vote_average'].notnull()))][['title', 'year', 'vote_count', 'vote_average', 'popularity']]

    qualified['vote_count'] = qualified['vote_count'].astype('int')
    qualified['vote_average'] = qualified['vote_average'].astype('int')

    qualified['wr'] = qualified.apply(lambda x: (x['vote_count']/(x['vote_count']+m) * x['vote_average']) + (m/(m+x['vote_count']) * C), axis=1)
    qualified = qualified.sort_values('wr', ascending = False).head(250)
    
    return qualified

이제 우리는 Top 15 로맨스 영화를 찾아주는 우리의 method를 보자.
(로맨스는 가장 유명한 영화 장르임에도 불구하고 Generic Top Chart에서는 거의 보이지 않았습니다.)

### **Top Romance Movies**
우리의 측정에 따르면 가장 유명한 영화는 Bollywood의 **Dilwale Dulhania Le Jayenge**이다. (힌디영화계의 용감한 자가 미인을 얻는다.)

In [18]:
build_chart('Romance').head(15)

,title,year,vote_count,vote_average,popularity,wr
10309,Dilwale Dulhania Le Jayenge,1995,661,9,34.457024,8.565285
351,Forrest Gump,1994,8147,8,48.307194,7.971357
876,Vertigo,1958,1162,8,18.20822,7.811667
40251,Your Name.,2016,1030,8,34.461252,7.789489
883,Some Like It Hot,1959,835,8,11.845107,7.745154
1132,Cinema Paradiso,1988,834,8,14.177005,7.744878
19901,Paperman,2012,734,8,7.198633,7.713951
37863,Sing Street,2016,669,8,10.672862,7.689483
882,The Apartment,1960,498,8,11.994281,7.599317
38718,The Handmaiden,2016,453,8,16.727405,7.566166


### Content Based Recommender
우리가 만든 위의 Recommender는 몇가지 한계가 있다.
그 중 하나는, 유저의 개인척 취향이 고려되지 않은 모두에게 같은 추천이라는 점이다. 
로맨틱한 영화를 좋아하고 액션을 싫어하는 사람이 Top 15 Chart를 본다면 아마도 대부분의 영화를 좋아하지 않을 것입니다. 그가 한 단계 더 나아가 장르별로 차트를 살펴본다면 여전히 최고의 추천을 받진 못할 것입니다.

* 우리가 해야할 일 : 처음에는 분야별로의 최상위를 추천해주고 (요기서 미리 함수가 짜여져 있어야할 것), 서비스를 사용할 수록 개인에게 맞는 추천


예를 들어, Dilwale Dulhania Le Jayenge, My Name is Khan and Kabhi Khushi Kabhi Gham을 좋아하는 사람을 생각해보자. 우린 그 사람이 Shahrukh Khan이라는 배우와 Karan Johar라는 감독을 좋아함을 유추할 수 있다. 심지어 그는좋아함을 유추할 수 있다. 그럼에도 불구하고 그는 로맨스 차트를 접속하였을 때, Top recommendation에서 이것들을 찾을 수 없었을 것이다.


우리의 추천을 좀 더 개인화시키기 위해, 나는 특정한 측정을 기반으로 한 영화 사이의 유사성을 계산해주는 engine을 개발하여 그 유저가 좋아하는 특별한 영화와 가장 비슷한 영화를 추천해줄 것이다. 이 engine을 개발하기 위해 영화 metadata (or content)를 쓸 것이기 때문에, 이것은 **Content Based Filtering**이라고 알려져 있다.


나는 두 개의 Content Based Recommender를 개발할 것이다. 아래의 기반을 둔 :
* 영화의 개요와 짧은 요약(Overview and Tagline)
* 영화의 캐스팅, 출연자, 키워드, 장르(Cast, Crew, Keywords and Genre)


또한 소개에 언급했다싶이, 사용할 수 있는 컴퓨팅 성능이 제한되어 있으므로 모든 영화의 일부를 사용할 것이다.

In [19]:
links_small = pd.read_csv('../input/the-movies-dataset/links_small.csv')
links_small = links_small[links_small['tmdbId'].notnull()]['tmdbId'].astype('int')

In [20]:
md = md.drop([19730, 29503, 35587])
md['id'] = md['id'].astype('int')
smd = md[md['id'].isin(links_small)]
smd.shape

(9099, 25)

우리는 original dataset인 45000개의 영화보다 5배 작은 9099개의 영화 데이터 셋을 가지고 있음


### 추천에 기반한 영화 서술
우리는 처음으로 영화 description과 tagline들을 이용한 추천 시스템을 개발할 것입니다. 우리는 기계의 성능을 판단할 수 있는 정량적 지표가 없기에 질적으로 수행되어야합니다.

In [21]:
# 내용을 기반으로 추천하기 위해 정보를 합쳐줌
smd['tagline'] = smd['tagline'].fillna('')
smd['description'] = smd['overview'] + smd['tagline']
smd['description'] = smd['description'].fillna('')

In [22]:
tf = TfidfVectorizer(analyzer='word', ngram_range=(1, 2), min_df=0, stop_words='english')
tfidf_matrix = tf.fit_transform(smd['description'])

In [23]:
tfidf_matrix.shape

(9099, 268124)

### Cosine Similarity
나는 두 영화 사이의 유사도의 수치적 정도를 계산하기 위해 Cosine Similarity를 사용할 것이다. 수학적으로, 이것은 아래와 같이 정의된다 :  
$ cosine(x, y) = \frac{x \cdot y^\top}{\parallel x \parallel \cdot \parallel y \parallel} $  
우리가 TF_IDF Vectorizer를 사용했기 때문에, 내적이 바로 Cosine Similarity Score를 제공해준다. 그러므로 우리는 cosine_similarities대신에 더 빠른 sklearn의 linear_kernel을 사용할 것이다.

In [24]:
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [25]:
cosine_sim[0]

array([1.        , 0.00680476, 0.        , ..., 0.        , 0.00344913,
       0.        ])

우리는 우리의 dataset 안의 모든 영화에 대한 쌍별 consine similarity matrix를 가지고 있습니다. 다음 단계는 cosine similarity 점수를 기반으로 가장 비슷한 영화 30개를 반환해주는 함수를 작성하겠습니다.

In [26]:
smd = smd.reset_index() # 이 과정은 왜 필요하지?
titles = smd['title']
indices = pd.Series(smd.index, index=smd['title'])

In [27]:
def get_recommendations(title):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:31]
    movie_indices = [i[0] for i in sim_scores]
    return titles.iloc[movie_indices]

모두 준비가 됐다. 이제 몇 가지 영화에 대해 top recommendation을 시도해보고 얼마나 좋은 지 확인해보자.

* 처음 접속한 유저 : 분류에 따라 상위 랭크를 보여주고(simple),
* 이용 중 : 조회수가 가장 높은 공모전과 비슷한 것을 보여주고(content based),
* 마지막 : 전체 데이터 기반으로 보여줌

In [28]:
get_recommendations('The Godfather').head(10)

973      The Godfather: Part II
8387                 The Family
3509                       Made
4196         Johnny Dangerously
29               Shanghai Triad
5667                       Fury
2412             American Movie
1582    The Godfather: Part III
4221                    8 Women
2159              Summer of Sam
Name: title, dtype: object

In [29]:
get_recommendations('The Dark Knight').head(10)

7931                      The Dark Knight Rises
132                              Batman Forever
1113                             Batman Returns
8227    Batman: The Dark Knight Returns, Part 2
7565                 Batman: Under the Red Hood
524                                      Batman
7901                           Batman: Year One
2579               Batman: Mask of the Phantasm
2696                                        JFK
8165    Batman: The Dark Knight Returns, Part 1
Name: title, dtype: object

The Dark Knight의 경우 시스템이 배트맨 영화로 식별하고 이후에 다른 배트맨 영화를 Top recommendation으로 추천할 수 있다. 하지만 안타깝게도 이것이 이 시스템이 현재 할 수 있는 전부다. 이것은 영화의 인기와 평점을 결정하는 출연진, 제작진, 감독 및 장르와 같이 매우 중요한 요소를 고려하지 않기 때문에 대부분의 사람들에게 별로 유용하지 않다. The Dark Knight를 좋아하는 사람은 아마도 Nolan 때문에 그것을 더 좋아할 것이고, Batman Forever와 모든 다른 수준 이하의 Batman 프랜차이즈 영화를 좋아하진 않을 것다. 


그러므로 우리는 Overview와 Tagline보다 훨씬 더 암시적인 metadata를 사용할 것입니다. 다음 섹션에서는, 우리는 장르와, 출연진 그리고 제작진을 고려요소로 채택해 더 수준 높은 추천 시스템을 개발할 것이다.


### Metadata Based Recommender
표준 메타데이터 기반 콘텐츠 추천기를 개발하기 위해, 우리는 현재 데이터를 crew와 keyword 데이터와 합칠 필요가 있다. 첫 번째로 이 데이터부터 준비해보자

In [30]:
credits = pd.read_csv('../input/the-movies-dataset/credits.csv')
keywords = pd.read_csv('../input/the-movies-dataset/keywords.csv')

In [31]:
keywords['id'] = keywords['id'].astype('int')
credits['id'] = credits['id'].astype('int')
md['id'] = md['id'].astype('int')

In [32]:
md.shape

(45463, 25)

In [33]:
md = md.merge(credits, on = 'id')
md = md.merge(keywords, on = 'id')


In [34]:
links_small

0          862
1         8844
2        15602
3        31357
4        11862
         ...  
9120    402672
9121    315011
9122    391698
9123    137608
9124    410803
Name: tmdbId, Length: 9112, dtype: int64

In [35]:
smd = md[md['id'].isin(links_small)]
smd.shape

(9219, 28)

우리는 이제 하나의 데이터프레임에 출연진, 제작진, 장르 및 credit이 들어가 있다. 아래의 직관을 통해 조금 더 사용가능하게 이야기 해보자

1. **Crew** : crew에서 다른 것들은 영화의 feel에 영향을 끼치지 못함으로 감독 데이터만 뽑아내겠다.
2. **Cast** : Cast를 선택하는 건 조금 더 까다롭다. 덜 알려진 배우와 마이너한 역할은 실제로도 영화에 대한 사람들의 의견에 영향을 끼치지 못한다. 그러므로, 우리는 메이저 캐릭터와 그것들의 배우만 뽑아야한다. 임의로 우리는 credits 리스트에서 나타나는 top3 배우만 고를 것이다

In [36]:
smd['cast'] = smd['cast'].apply(literal_eval)
smd['crew'] = smd['crew'].apply(literal_eval)
smd['keywords'] = smd['keywords'].apply(literal_eval)

In [37]:
smd['cast_size'] = smd['cast'].apply(lambda x : len(x))
smd['crew_size'] = smd['crew'].apply(lambda x : len(x))

In [38]:
def get_director(x) : 
    for i in x :
        if i['job'] == 'Director' :
            return i['name']
    return np.nan

In [39]:
smd['director'] = smd['crew'].apply(get_director)

In [40]:
smd['cast'] = smd['cast'].apply(lambda x : [i['name'] for i in
            x] if isinstance(x, list) else [])

smd['cast'] = smd['cast'].apply(lambda x : x[:3] if len(x) >= 3
                               else x)

In [41]:
smd['keywords'] = smd['keywords'].apply(lambda x : [i['name'] for i in
            x] if isinstance(x, list) else [])

추천 시스템을 개발하는 대 접근 방식은 극도로 진부하다. 내가 할 계획은 장르, 감독, 주연 및 키워드로 구성된 모든 영화에 대한 데이터 덤프를 만드는 것이다. 나는 Description Recomment에서 했던 것처럼 **Count Vectorizer**을 사용하여 우리의 count matrix를 개발할 것이다. 이제 남은 단계는 우리가 이전에 했던 것과 유사합니다. cosine similarity를 계산하고 가장 유사한 영화를 반환합니다.

다음은 장르 및 크레딧 데이터를 준비할 때 따르는 단계입니다.
1. 모든 요소에서 **공백을 제거하고 소문자로 변환**합니다. 이렇게 하면, 우리의 엔진은 Johnny Depp과 Johnny Galecki를 혼동하지 않습니다.
2. **감독을 3번 언급**하여 전체 출연진에 비해 더 가중치를 부여합니다.

In [42]:
smd['director'] = smd['director'].astype('str').apply(lambda x: str.lower(x.replace(" ","")))
smd['director'] = smd['director'].apply(lambda x: [x, x, x])

### Keywords
키워드를 사용하기 전에 소량의 전처리를 수행하겠다. 처음으로, 데이터셋에 모든 keyword의 빈도를 계산한다.

In [43]:
s = smd.apply(lambda x: pd.Series(x['keywords']), axis = 1).stack().reset_index(level = 1, drop=True)
s.name = 'keyword'

In [44]:
s = s.value_counts()
s[:5]

independent film        610
woman director          550
murder                  399
duringcreditsstinger    327
based on novel          318
Name: keyword, dtype: int64

키워드는 1에서 610번까지의 빈도로 발생한다. 한번만 발생한 키워드는 사용하지 않겠다. 따라서 안전하게 제거할 수 있다. 마지막으로 Dogs와 Dog같은 같은 단어가 동일하게 간주되도록 모든 단어를 어간으로 변환한다.

In [45]:
s = s[s>1]

In [46]:
stemmer = SnowballStemmer('english')
stemmer.stem('dogs')

'dog'

In [47]:
def filter_keywords(x) :
    words = []
    for i in x:
        if i in s:
            words.append(i)
    return words

In [48]:
smd['keywords'] = smd['keywords'].apply(filter_keywords)
smd['keywords'] = smd['keywords'].apply(lambda x: [stemmer.stem(i) for i in x])
smd['keywords'] = smd['keywords'].apply(lambda x: [str.lower(i.replace(" ", "")) for i in x])

In [49]:
smd['soup'] = smd['keywords'] + smd['cast'] + smd['director'] + smd['genres']
smd['soup'] = smd['soup'].apply(lambda x: ' '.join(x))

In [50]:
count = CountVectorizer(analyzer = 'word', ngram_range = (1, 2),
                        min_df = 0,stop_words='english')

count_matrix = count.fit_transform(smd['soup'])

In [51]:
cosine_sim = cosine_similarity(count_matrix, count_matrix)

In [52]:
smd = smd.reset_index()
titles = smd['title']
indices = pd.Series(smd.index, index=smd['title'])

우리는 이전에 만들었던 get_recommendation 함수를 다시 사용할 것이다. 우리의 cosine similarity score가 변경됐기 때문에, 다르고 아마 더 좋은 결과를 반환해줄 것이다. The Dark Knight를 다시 검증해보고 추천이 무엇인지 확인해보자.

In [53]:
get_recommendations('The Dark Knight').head(10)

8031         The Dark Knight Rises
6218                 Batman Begins
6623                  The Prestige
2085                     Following
7648                     Inception
4145                      Insomnia
3381                       Memento
8613                  Interstellar
1134                Batman Returns
7659    Batman: Under the Red Hood
Name: title, dtype: object

이번에 얻은 결과는 좀 더 만족스럽다. 추천 시스템은 다른 Christopher Nolan 영화들을 인식하고 그것을 Top 추천에 놓은 것으로 보인다. (감독에 준 높은 weight때문에) 나는 The Dark Knight 뿐아니라 리스트에 있는 Batman Begins, The Prestige 그리고 The Dark Kngiht Rises를 포함한 다른 것들도 보면서 즐거웠다.


우리는 물론 요소(감독, 배우, 장르)에 다른 weight를 시도하고, soup에서 사용되는 keyword의 수를 제한하고, 같은 언어 등의 영화를 보여주며 빈도에 따라 장르의 가중치를 부여할 수 있습니다.

Popularity와 Ratings
우리의 추천 시스템에서 주목해야할 점 중 하나는 rating과 popularity에 관계 없이 추천해준다는 것이다. **Batman and Robin**은 **The Dark Kngiht**와 꽤 많이 비슷한 특성을 가지고 있지만 누구에게서도 추천받지 못하는 끔찍한 영화다.

그러므로, 우리는 나쁜 영화를 제거하고 유명하고 좋은 비평을 받은 작품을 반환하는 메커니즘을 추가할 것이다.

유사도 점수와 영화의 60%의 vote를 계산하여 Top25 영화를 가져올 것이다. 그런 다음 이것을 M의 값으로 사용하여 Simple Recommender 섹션에서 했던 것처럼 IMDB의 공식을 사용하여 각 영화의 가중치 등급을 계산한다.

In [54]:
idx = indices['The Dark Knight']
sim_scores = list(enumerate(cosine_sim[idx]))
sim_scores = sorted(sim_scores, key = lambda x: x[1], reverse = True)
sim_scores = sim_scores[1:26]
movie_indices = [i[0] for i in sim_scores]

In [55]:
def improved_recommendations(title): 
    movies = smd.iloc[movie_indices][['title', 'vote_count', 'vote_average', 'year']]
    vote_counts = movies[movies['vote_count'].notnull()]['vote_count'].astype('int')
    vote_averages = movies[movies['vote_average'].notnull()]['vote_average'].astype('int')
    C = vote_averages.mean()
    m = vote_counts.quantile(0.6)
    qualified = movies[(movies['vote_count']>=m)&(movies['vote_count'].notnull())&
                (movies['vote_average'].notnull())]
    qualified['vote_count'] = qualified['vote_count'].astype('int')
    qualified['vote_average'] = qualified['vote_average'].astype('int')
    qualified['wr'] = qualified.apply(weighted_rating, axis = 1)
    qualified = qualified.sort_values('wr', ascending = False).head(10)
    
    return qualified

In [56]:
improved_recommendations('The Dark Knight')

,title,vote_count,vote_average,year,wr
7648,Inception,14075,8,2010,7.917588
8613,Interstellar,11187,8,2014,7.897107
6623,The Prestige,4510,8,2006,7.758148
3381,Memento,4168,8,2000,7.740175
8031,The Dark Knight Rises,9263,7,2012,6.921448
6218,Batman Begins,7511,7,2005,6.904127
524,Batman,2145,7,1989,6.704647
6622,Children of Men,2120,7,2006,6.701756
8419,Man of Steel,6462,6,2013,5.952478
9024,Batman v Superman: Dawn of Justice,7189,5,2016,5.013943


불행하게도, **Batman and Robin**은 우리의 추천 목록에서 사라지지 않았다.
이는 TMDB에서 평균보다 약간 낮은 4등급이라는 사실 때문일 것이다. 이것은 확실히 이 놀라운 **The Dark Knight**가 오직 7점뿐이기에 4점을 받을 자격이 없다.
 하지만 우리가 이것에 대해 할 수 있는게 없다. 그러므로, 우리는 Content Based Recommender 섹션을 여기서 마치고 hybrid engine을 만들 때 돌아오자

## Collaborative Filtering
우리의 content based engine은 몇가지 제한을 가지고 있다. 특정 영화와 가까운 영화만 제한할 수 있다. 즉, 취향을 포착하고 장르 전반에 걸쳐 추천을 제공할 수 없다. 


또한, 우리가 만든 그 engine은 사용자의 개인적인 취향과 편향을 포착하지 않는다는 점에서 실제로도 개인적인 것이 아니다. 영화에 기반을 한 추천을 위해 엔진을 쿼리하는 사람은 그 사람이 누구인지에 대해 관계 없이 해당 영화에 대한 동일한 추천을 받게 된다.

그러므로, 이번 섹션에서 우리는 **Collaborative Filtering**이라고 불리는 기술을 사용하여 영화 관람객들에게 추천을 해보자. Collaborative Filtering은 나와 비슷한 사용자가 사용 / 경험한 특정 제품 또는 서비스를 얼마나 좋아할 지 예측하는 데 사용할 수 있다는 생각을 기반으로 만들어졌다. 

처음부터 Collaborative Filtering을 구현하지는 않을 것이다. 대신 RMSE를 최소화시켜주고 훌륭한 추천을 주는 **SVD**와 같은 강력한 알고리즘을 사용한 **Surprise** library를 사용할 것이다.

In [57]:
reader = Reader()

In [58]:
ratings = pd.read_csv('../input/the-movies-dataset/ratings_small.csv')
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [59]:
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)

svd = SVD()
cross_validate(svd, data, measures=['RMSE', 'MAE'], cv=10, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 10 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Fold 6  Fold 7  Fold 8  Fold 9  Fold 10 Mean    Std     
RMSE (testset)    0.8914  0.8918  0.8861  0.8835  0.8886  0.9036  0.8903  0.8947  0.8865  0.8931  0.8910  0.0053  
MAE (testset)     0.6890  0.6843  0.6814  0.6788  0.6824  0.6895  0.6890  0.6883  0.6882  0.6869  0.6858  0.0036  
Fit time          9.95    10.12   9.94    9.91    9.99    9.96    10.01   10.03   9.98    9.94    9.98    0.06    
Test time         0.11    0.10    0.10    0.10    0.10    0.10    0.11    0.10    0.10    0.10    0.10    0.00    


{'test_rmse': array([0.89138804, 0.89183635, 0.88614623, 0.88352331, 0.88861029,
        0.90359918, 0.8902604 , 0.89470082, 0.88647499, 0.89312209]),
 'test_mae': array([0.68901444, 0.68430754, 0.68142662, 0.67884031, 0.68243101,
        0.68948616, 0.6890358 , 0.68830809, 0.68823265, 0.68685566]),
 'fit_time': (9.953301191329956,
  10.123114347457886,
  9.944744110107422,
  9.90552568435669,
  9.991703987121582,
  9.960370779037476,
  10.006992101669312,
  10.029311180114746,
  9.980653285980225,
  9.941281080245972),
 'test_time': (0.10543346405029297,
  0.10295701026916504,
  0.10406255722045898,
  0.10489201545715332,
  0.1047518253326416,
  0.10451936721801758,
  0.10583281517028809,
  0.1037759780883789,
  0.10432934761047363,
  0.1043701171875)}

우리는 RMSE 0.9020를 얻었다. 이는 우리의 케이스에 충분하다. 우리의 데이터셋을 훈련시키고 예측해보자

In [60]:
trainset = data.build_full_trainset()
svd.fit(trainset)

이제 유저를 골라서 그가 준 rating을 체크해보자

In [61]:
ratings[ratings['userId'] == 1]

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205
5,1,1263,2.0,1260759151
6,1,1287,2.0,1260759187
7,1,1293,2.0,1260759148
8,1,1339,3.5,1260759125
9,1,1343,2.0,1260759131


In [62]:
svd.predict(1, 302, 3)

Prediction(uid=1, iid=302, r_ui=3, est=2.568277351069606, details={'was_impossible': False})

Movie ID 302에 대해, 우리는 추정치 2.51을 예측했다. 이 추천 시스템의 놀라운 기능 중 하나는 영화가 무엇인지 (또는 포함 된 내용) 상관하지 않는다는 것입니다. 순전히 할당 된 영화 ID를 기반으로 작동하며 다른 사용자가 어떻게 영화를 매겼는지에 따라 예측하려고 합니다.

## Hybrid Recommender

이번 섹션에서는, 이전에 시도했던 content based와 collaboraitve filter based 엔진을 이용한 기술들로 간단한 hybrid 추천 시스템을 개발할 것이다.
이것은 작업이 어떻게 이루어지는가이다 : 
* **Input** : User ID와 영화의 제목
* **Output** : 특정한 유저에 예상 등급을 기준으로 유사한 영화의 정렬

In [63]:
def convert_int(x):
    try:
        return int(x)
    
    except:
        return np.nan

In [64]:
id_map = pd.read_csv('../input/the-movies-dataset/links_small.csv')[['movieId', 'tmdbId']]
id_map['tmdbId'] = id_map['tmdbId'].apply(convert_int)
id_map.columns = ['movieId', 'id']
id_map = id_map.merge(smd[['title', 'id']], on = 'id').set_index('title')

In [65]:
indices_map = id_map.set_index('id')

In [66]:
def hybrid(userId, title):
    idx = indices[title]
    tmdbId = id_map.loc[title]['id']
    movie_id = id_map.loc[title]['movieId']
    
    # content based
    sim_scores = list(enumerate(cosine_sim[int(idx)]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:26]
    movie_indices = [i[0] for i in sim_scores]
        
    movies = smd.iloc[movie_indices][['title', 'vote_count', 'vote_average', 'year', 'id']]
    # collaborative
    movies['est'] = movies['id'].apply(lambda x: svd.predict(userId, indices_map.loc[x]['movieId']).est)
    movies = movies.sort_values('est', ascending=False)
    return movies.head(10)

In [67]:
hybrid(1, 'Avatar')

,title,vote_count,vote_average,year,id,est
8712,Guardians of the Galaxy,10014.0,7.9,2014,118340,3.251037
1011,The Terminator,4208.0,7.4,1984,218,3.116204
974,Aliens,3282.0,7.7,1986,679,3.045930
8658,X-Men: Days of Future Past,6155.0,7.5,2014,127585,3.031497
8401,Star Trek Into Darkness,4479.0,7.4,2013,54138,3.001068
987,Alien,4564.0,7.9,1979,348,2.959553
522,Terminator 2: Judgment Day,4274.0,7.7,1991,280,2.856413
1668,Return from Witch Mountain,38.0,5.6,1978,14822,2.735318
344,True Lies,1138.0,6.8,1994,36955,2.733984
922,The Abyss,822.0,7.1,1989,2756,2.668469


In [68]:
hybrid(500, 'Avatar')

,title,vote_count,vote_average,year,id,est
8712,Guardians of the Galaxy,10014.0,7.9,2014,118340,3.407659
1376,Titanic,7770.0,7.5,1997,597,3.399113
6084,Beastmaster 2: Through the Portal of Time,17.0,4.6,1991,27549,3.393960
8401,Star Trek Into Darkness,4479.0,7.4,2013,54138,3.319184
8658,X-Men: Days of Future Past,6155.0,7.5,2014,127585,3.306021
974,Aliens,3282.0,7.7,1986,679,3.290333
1011,The Terminator,4208.0,7.4,1984,218,3.275382
987,Alien,4564.0,7.9,1979,348,3.198941
1668,Return from Witch Mountain,38.0,5.6,1978,14822,3.012842
3060,Sinbad and the Eye of the Tiger,39.0,6.3,1977,11940,2.996728


우리의 hybrid recommender에 의해, 같은 영화더라도 다른 유저일 경우 다른 추천을 받음을 알 수 있었다. 따라서 우리의 추천 시스템은 좀 더 개인적이고 특정한 유저에게 딱 맞는다.

### 결론
이 노트북에서, 나는 4가지의 다른 아이디어와 알고리즘을 기반으로 한 추천 engine을 만들었다. 다음과 같다 :
1. **Simple Recommender** : 이 시스템은 일반적이고, 특별한 장르의 Top Movie Chart를 위해 전반적인 TMDB 투표수와 투표 평균을 이용했다. IMDB 가중치 평가 시스템은 저열ㄹ이 최종적으로 수행된 평가를 계산하는 데 사용되었다.

2. **Content Based Recommender** : 우리는 2가지의 content based engine을 개발하였다. 하나는 예측에 영화 overview와 tagline들을 input으로 사용하였고, 다른 것은 metadata로 cast, crew, genre, keyword를 사용하였다. 우리는 또한 더 많은 투표와 더 높은 등급의 영화를 더 선호하도록 간단한 필터를 장치했다.

3. **Collaborative Filtering** : 우리는 강력한 Surprise Library를 사용하여 SVD를 기반으로 한 collaborative filter를 개발하였다. RMSE는 1을 넘지 않았고, engine은 주어진 유저와 영화에 대한 추정 등급을 주었다.

4. **Hybrid Engine** : content와 collaborative filtering의 아이디어를 모아 유저에 대해 내부적으로 계산한 추정 등급을 기반으로 특정 사용자에게 영화 추천을 제공하는 엔진을 구축하였다.

## ---끝---